In [19]:
!pip install selenium

## 

In [150]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# ✅ ドライバ初期化
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

# ✅ ログインページへアクセス
driver.get("https://hotel-example-site.takeyaqa.dev/ja/")
driver.find_element(By.LINK_TEXT, "ログイン").click()

# ✅ ログインフォーム入力
wait.until(EC.presence_of_element_located((By.ID, "email")))
driver.find_element(By.ID, "email").send_keys("ichiro@example.com")
driver.find_element(By.ID, "password").send_keys("password")
driver.find_element(By.ID, "login-button").click()
time.sleep(1.5)

# ✅ モーダル（ポップアップ）が出ていれば閉じる
try:
    buttons = driver.find_elements(By.CLASS_NAME, "swal-button--confirm")
    if buttons:
        driver.execute_script("arguments[0].click();", buttons[0])
        print("✅ モーダルOKボタンをクリックしました")
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, "swal-modal"))
        )
except Exception as e:
    print("⚠️ モーダルは表示されなかったか、既に閉じられていました")

# ✅ マイページ表示確認
wait.until(EC.presence_of_element_located((By.LINK_TEXT, "マイページ")))
print("✅ ログイン成功")

# ✅ プラン一覧ページへ直接アクセス（描画バグ回避）
driver.get("https://hotel-example-site.takeyaqa.dev/ja/plans")



✅ ログイン成功


In [151]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

wait = WebDriverWait(driver, 10)

# 「テーマ優待プラン」のカードを取得
plans = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "card")))

for plan in plans:
    if "テーマパーク優待プラン" in plan.text:
        # プランの中の「このプランで予約」ボタンを探してクリック
        reserve_button = plan.find_element(By.LINK_TEXT, "このプランで予約")
        reserve_button.click()
        print("✅ テーマパーク優待プランを選択しました")
        break
else:
    print("❌ テーマパーク優待プランが見つかりませんでした")




# 👉 新しいタブが開くのを待って切り替える（最後に戻ってくるためにハンドルを記憶）
original_handle = driver.current_window_handle

wait.until(lambda driver: len(driver.window_handles) > 1)
driver.switch_to.window(driver.window_handles[-1])  # 一番最後のタブへ切り替え
print("✅ 新しいタブへ切り替えました")

# 新しいタブで日付フィールドを探す
wait.until(EC.presence_of_element_located((By.ID, "date")))
print("✅ 予約フォームの読み込みを確認しました")



# ✅ 予約ページへの遷移待ち（「date」フィールドが表示されるまで待つ）
wait.until(EC.presence_of_element_located((By.ID, "date")))

# --- 入力処理 ---

# 宿泊日
date_input = driver.find_element(By.ID, "date")
date_input.clear()
date_input.send_keys("2025/07/15")
date_input.send_keys(Keys.ESCAPE)
time.sleep(0.5)

# 宿泊数
driver.find_element(By.ID, "term").clear()
driver.find_element(By.ID, "term").send_keys("3")

# 人数
driver.find_element(By.ID, "head-count").clear()
driver.find_element(By.ID, "head-count").send_keys("2")

# 追加プラン：朝食バイキング
driver.find_element(By.ID, "breakfast").click()

# 連絡方法：電話
from selenium.webdriver.support.ui import Select

# ▼ ドロップダウン（連絡方法）を選択
select_element = driver.find_element(By.ID, "contact")  # IDが contact の<select>要素を想定
select = Select(select_element)

# 「電話でのご連絡」を選択
select.select_by_visible_text("電話でのご連絡")

# 電話番号
driver.find_element(By.ID, "tel").clear()
driver.find_element(By.ID, "tel").send_keys("00011112222")


# 合計金額の確認
wait.until(EC.presence_of_element_located((By.ID, "total-bill")))
price = driver.find_element(By.ID, "total-bill").text
print("✅ 合計金額:", price)

# 「予約を確認する」ボタンをクリック
driver.find_element(By.ID, "submit-button").click()
print("✅ 『宿泊予約確認』画面に遷移しました")


from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta

# 7. 予約確認画面に入力データが正しく表示されているかチェック（例）
# 入力した宿泊開始日と泊数

wait.until(EC.title_contains("宿泊予約確認"))
wait.until(EC.visibility_of_element_located((By.ID, "term")))
time.sleep(1)  # 動的更新待ち用

start_date_str = "2025/07/15"
nights = 3

# 日付文字列をdatetime型に変換
start_date = datetime.strptime(start_date_str, "%Y/%m/%d")
end_date = start_date + timedelta(days=nights)

# 確認画面の期間テキストを取得
confirm_period = driver.find_element(By.ID, "term").text

# 期待する文字列のフォーマット例（「2025年7月15日 〜 2025年7月18日 3泊」）
expected_str = f"{start_date.year}年{start_date.month}月{start_date.day}日 〜 {end_date.year}年{end_date.month}月{end_date.day}日 {nights}泊"

# アサートで一致チェック
assert expected_str in confirm_period, f"期間が一致しません: 期待値='{expected_str}', 実際='{confirm_period}'"

confirm_people = driver.find_element(By.ID, "head-count").text
assert "2" in confirm_people, "人数が一致しません"

# 追加プランの確認
confirm_plans = driver.find_element(By.ID, "plans").text
assert "朝食バイキング" in confirm_plans, f"追加プランが一致しません: 実際='{confirm_plans}'"

# お名前の確認
confirm_name = driver.find_element(By.ID, "username").text
assert "山田一郎様" == confirm_name, f"お名前が一致しません: 実際='{confirm_name}'"

# 確認のご連絡の確認
confirm_contact = driver.find_element(By.ID, "contact").text
assert "電話：00011112222" == confirm_contact, f"確認のご連絡が一致しません: 実際='{confirm_contact}'"

# ご要望・ご連絡事項等の確認
confirm_comment = driver.find_element(By.ID, "comment").text.strip()
assert "なし" == confirm_comment, f"ご要望・ご連絡事項等が一致しません: 実際='{confirm_comment}'"
# 8. 「この内容で予約する」ボタンをクリック

# 「この内容で予約する」ボタンをクリック
driver.find_element(By.XPATH, "//button[text()='この内容で予約する']").click()

# モーダルの「閉じる」ボタンが表示されるまで待つ
wait.until(EC.visibility_of_element_located((By.ID, "success-modal")))

# 「閉じる」ボタンをクリック
driver.find_element(By.XPATH, "//div[@id='success-modal']//button[text()='閉じる']").click()

#11

# 現在のウインドウハンドル一覧と現在アクティブなハンドルを表示
print("ウインドウハンドル一覧:", driver.window_handles)
print("現在のウインドウハンドル:", driver.current_window_handle)

# ちゃんと最後のタブに切り替える例
driver.switch_to.window(driver.window_handles[-1])

# URLを取得
print("現在のURL:", driver.current_url)

# 処理が終わったら元のタブに戻る
driver.switch_to.window(original_handle)
print("✅ 元のタブに戻りました")

✅ テーマパーク優待プランを選択しました
✅ 新しいタブへ切り替えました
✅ 予約フォームの読み込みを確認しました
✅ 合計金額: 66,000円
✅ 『宿泊予約確認』画面に遷移しました
ウインドウハンドル一覧: ['67CDE2C177B07167A3A94195B777DFEA', 'A9E1AD48DC5AE59657C2F876DEB75477']
現在のウインドウハンドル: A9E1AD48DC5AE59657C2F876DEB75477
現在のURL: https://hotel-example-site.takeyaqa.dev/ja/confirm.html
✅ 元のタブに戻りました
